# TRANSFORMACIÓN DE DATOS

## IMPORTAR PAQUETES

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from category_encoders import TargetEncoder
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import Binarizer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import QuantileTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MaxAbsScaler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

#Automcompletar rápido
%config IPCompleter.greedy=True

## IMPORTAR LOS DATOS

1.- Sustituir la ruta del proyecto.

In [6]:
ruta_proyecto = 'C:/Users/pelop/OneDrive/Desktop/Curso Data Science Pedro/2 CURSO DATA SCIENCE/03_MACHINE_LEARNING/07_CASOS/01_LEADSCORING'

2.- Nombrar los ficheros de datos.

In [7]:
nombre_cat = 'cat_resultado_eda.pickle'
nombre_num = 'num_resultado_eda.pickle'

3.- Cargar los datos.

In [8]:
cat = pd.read_pickle(ruta_proyecto + '/02_Datos/03_Trabajo/' + nombre_cat).reset_index(drop = True)
num = pd.read_pickle(ruta_proyecto + '/02_Datos/03_Trabajo/' + nombre_num).reset_index(drop = True)

4.- Separar la target.

In [9]:
num.head()

,id,compra,visitas_total,tiempo_en_site_total,paginas_vistas_visita,score_actividad,score_perfil
0,660737,0,0,0,0.0,15.0,15.0
1,660728,0,5,674,2.5,15.0,15.0
2,660727,1,2,1532,2.0,14.0,20.0
3,660719,0,1,305,1.0,13.0,17.0
4,660673,1,2,1640,2.0,14.0,20.0


In [10]:
target = num[['compra']].copy().reset_index(drop=True)

In [11]:
for i in cat.columns.to_list():
    print(i)

id
origen
fuente
ult_actividad
ambito
ocupacion
descarga_lm


In [12]:
for i in num.columns.to_list():
    print(i)

id
compra
visitas_total
tiempo_en_site_total
paginas_vistas_visita
score_actividad
score_perfil


* Para las numéricas no vamos a hacer ni **discretización** ni **binarización**, ya que no estamos buscando ningún tipo de generalización ni de interpretación en este caso. Solo estamos buscando capacidad predictiva. Discretizar es útil cuando a posteriori queremos dar una explicación a los resultados. Y como vamos a realizar una segmentación (**KMeans**) no tiene tampoco mucho sentido discretizar.
* Tampoco vamos a hacer la **normalización** pues ya vimos en el EDA que las variables numéricas eran bastante normales (distribución gaussiana).
* Por otro lado sí tenemos que hacer **reescalado** pues el algoritmo de KMeans es muy sensible a la escala. **MinMaxScaling** es la opción más razonable para este caso ya que como tenemos muchas variables categóricas con OHE (entre 0 y 1), minmaxscaling también nos las va a dejar en este formato. Además como vamos a hacer un cluster es aún más determinante la escala.

## TRANSFORMACIÓN DE CATEGÓRICAS

In [13]:
cat.origen.value_counts() # Es una variable nominal -> OHE

origen
Landing Page Submission    3081
API                        1522
Lead Add Form               236
OTROS                        34
Name: count, dtype: int64

In [14]:
cat.fuente.value_counts() # Es una variable nominal -> OHE

fuente
Google            1878
Direct Traffic    1567
Organic Search     752
Chat               307
Reference          222
OTROS              147
Name: count, dtype: int64

In [15]:
cat.ult_actividad.value_counts() # Es una variable nominal -> OHE

ult_actividad
Email Opened               2014
SMS Sent                   1516
Page Visited on Website     381
Converted to Lead           302
Chat Conversation           276
OTROS                       237
Email Link Clicked          147
Name: count, dtype: int64

In [16]:
cat.ambito.value_counts() # Es una variable nominal -> OHE

ambito
Select                               1226
Finance Management                    609
Human Resource Management             518
Marketing Management                  510
OTROS                                 383
Operations Management                 335
Business Administration               254
Banking, Investment And Insurance     227
IT Projects Management                224
Supply Chain Management               222
Media and Advertising                 139
Travel and Tourism                    116
International Business                110
Name: count, dtype: int64

In [17]:
cat.ocupacion.value_counts() # Esta está en la zona gris, pues podríamos decir que OTROS < Student < Unemployed < Working Professional
                             # Pero como tampoco está muy claro que es mejor la dejamos como nominal y no ordinal -> OHE

ocupacion
Unemployed              4329
Working Professional     421
Student                  100
OTROS                     23
Name: count, dtype: int64

In [18]:
cat.descarga_lm.value_counts() # Es una variable nominal -> OHE

descarga_lm
No     3035
Yes    1838
Name: count, dtype: int64

### One Hot Encoding

#### Variables a aplicar OHE

In [19]:
var_ohe = ['origen','fuente','ult_actividad','ambito','ocupacion','descarga_lm']

#### Instanciar

In [20]:
ohe = OneHotEncoder(sparse_output = False, handle_unknown='ignore')

#### Entrenar y aplicar

In [21]:
cat_ohe = ohe.fit_transform(cat[var_ohe])

#### Guardar como dataframe

In [22]:
cat_ohe = pd.DataFrame(cat_ohe, columns = ohe.get_feature_names_out())
cat_ohe

,origen_API,origen_Landing Page Submission,origen_Lead Add Form,origen_OTROS,fuente_Chat,fuente_Direct Traffic,fuente_Google,fuente_OTROS,fuente_Organic Search,fuente_Reference,...,ambito_Operations Management,ambito_Select,ambito_Supply Chain Management,ambito_Travel and Tourism,ocupacion_OTROS,ocupacion_Student,ocupacion_Unemployed,ocupacion_Working Professional,descarga_lm_No,descarga_lm_Yes
0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4868,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
4869,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
4870,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
4871,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


## TRANSFORMACIÓN DE NUMÉRICAS

No hay transformaciones que aplicar a las variables numéricas en este caso

## UNIFICAR DATASETS TRANSFORMADOS

In [23]:
df = pd.concat([cat_ohe,num], axis=1)

## REESCALAR VARIABLES

### Con Min-Max

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4873 entries, 0 to 4872
Data columns (total 43 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   origen_API                                4873 non-null   float64
 1   origen_Landing Page Submission            4873 non-null   float64
 2   origen_Lead Add Form                      4873 non-null   float64
 3   origen_OTROS                              4873 non-null   float64
 4   fuente_Chat                               4873 non-null   float64
 5   fuente_Direct Traffic                     4873 non-null   float64
 6   fuente_Google                             4873 non-null   float64
 7   fuente_OTROS                              4873 non-null   float64
 8   fuente_Organic Search                     4873 non-null   float64
 9   fuente_Reference                          4873 non-null   float64
 10  ult_actividad_Chat Conversation     

#### Variables a reescalar con Min-Max

In [25]:
var_mms = df.iloc[:,38:].columns
var_mms

Index(['visitas_total', 'tiempo_en_site_total', 'paginas_vistas_visita',
       'score_actividad', 'score_perfil'],
      dtype='object')

#### Instanciar

In [26]:
mms = MinMaxScaler()

#### Entrenar y aplicar

In [27]:
df_mms = mms.fit_transform(df[var_mms])

#### Guardar como dataframe

In [28]:
#Añadir sufijos a los nombres
nombres_mms = [variable + '_mms' for variable in var_mms]

#Guardar como dataframe
df_mms = pd.DataFrame(df_mms,columns = nombres_mms)
df_mms

,visitas_total_mms,tiempo_en_site_total_mms,paginas_vistas_visita_mms,score_actividad_mms,score_perfil_mms
0,0.00,0.000000,0.000,0.727273,0.444444
1,0.10,0.296655,0.125,0.727273,0.444444
2,0.04,0.674296,0.100,0.636364,1.000000
3,0.02,0.134243,0.050,0.545455,0.666667
4,0.04,0.721831,0.100,0.636364,1.000000
...,...,...,...,...,...
4868,0.16,0.447183,0.200,0.727273,1.000000
4869,0.04,0.779049,0.100,0.636364,1.000000
4870,0.10,0.092430,0.125,0.636364,1.000000
4871,0.04,0.104754,0.100,0.636364,0.888889


## UNIFICAR DATASETS REESCALADOS

In [29]:
id = df.id

In [30]:
pd.concat([id,cat_ohe,df_mms,target], axis = 1).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4873 entries, 0 to 4872
Data columns (total 43 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   id                                        4873 non-null   int64  
 1   origen_API                                4873 non-null   float64
 2   origen_Landing Page Submission            4873 non-null   float64
 3   origen_Lead Add Form                      4873 non-null   float64
 4   origen_OTROS                              4873 non-null   float64
 5   fuente_Chat                               4873 non-null   float64
 6   fuente_Direct Traffic                     4873 non-null   float64
 7   fuente_Google                             4873 non-null   float64
 8   fuente_OTROS                              4873 non-null   float64
 9   fuente_Organic Search                     4873 non-null   float64
 10  fuente_Reference                    

### Unir todos los dataframes en el tablón analítico

In [100]:
df_tablon = pd.concat([id,cat_ohe,df_mms,target], axis = 1).set_index('id')
df_tablon

,origen_API,origen_Landing Page Submission,origen_Lead Add Form,origen_OTROS,fuente_Chat,fuente_Direct Traffic,fuente_Google,fuente_OTROS,fuente_Organic Search,fuente_Reference,...,ocupacion_Unemployed,ocupacion_Working Professional,descarga_lm_No,descarga_lm_Yes,visitas_total_mms,tiempo_en_site_total_mms,paginas_vistas_visita_mms,score_actividad_mms,score_perfil_mms,compra
id,,,,,,,,,,,,,,,,,,,,,
660737,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.00,0.000000,0.000,0.727273,0.444444,0
660728,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,0.10,0.296655,0.125,0.727273,0.444444,0
660727,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.04,0.674296,0.100,0.636364,1.000000,1
660719,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.02,0.134243,0.050,0.545455,0.666667,0
660673,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.04,0.721831,0.100,0.636364,1.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
579697,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.16,0.447183,0.200,0.727273,1.000000,1
579642,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.04,0.779049,0.100,0.636364,1.000000,0
579615,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.10,0.092430,0.125,0.636364,1.000000,1


Comprobaciones:

In [97]:
df_tablon.isna().sum()

id                                          0
origen_API                                  0
origen_Landing Page Submission              0
origen_Lead Add Form                        0
origen_OTROS                                0
fuente_Chat                                 0
fuente_Direct Traffic                       0
fuente_Google                               0
fuente_OTROS                                0
fuente_Organic Search                       0
fuente_Reference                            0
ult_actividad_Chat Conversation             0
ult_actividad_Converted to Lead             0
ult_actividad_Email Link Clicked            0
ult_actividad_Email Opened                  0
ult_actividad_OTROS                         0
ult_actividad_Page Visited on Website       0
ult_actividad_SMS Sent                      0
ambito_Banking, Investment And Insurance    0
ambito_Business Administration              0
ambito_Finance Management                   0
ambito_Human Resource Management  

In [101]:
df_tablon.describe().T

,count,mean,std,min,25%,50%,75%,max
origen_API,4873.0,0.312333,0.463492,0.0,0.000000,0.000000,1.000000,1.0
origen_Landing Page Submission,4873.0,0.632259,0.482240,0.0,0.000000,1.000000,1.000000,1.0
origen_Lead Add Form,4873.0,0.048430,0.214695,0.0,0.000000,0.000000,0.000000,1.0
origen_OTROS,4873.0,0.006977,0.083246,0.0,0.000000,0.000000,0.000000,1.0
fuente_Chat,4873.0,0.063000,0.242988,0.0,0.000000,0.000000,0.000000,1.0
fuente_Direct Traffic,4873.0,0.321568,0.467126,0.0,0.000000,0.000000,1.000000,1.0
fuente_Google,4873.0,0.385389,0.486737,0.0,0.000000,0.000000,1.000000,1.0
fuente_OTROS,4873.0,0.030166,0.171062,0.0,0.000000,0.000000,0.000000,1.0
fuente_Organic Search,4873.0,0.154320,0.361292,0.0,0.000000,0.000000,0.000000,1.0
fuente_Reference,4873.0,0.045557,0.208544,0.0,0.000000,0.000000,0.000000,1.0


## GUARDAR DATASET TRAS TRANSFORMACIÓN DE DATOS

En formato pickle para no perder las modificaciones de metadatos.

In [102]:
#Definir los nombres del archivo
ruta_df_tablon = ruta_proyecto + '/02_Datos/03_Trabajo/' + 'df_tablon.pickle'

In [103]:
#Guardar los archivos
df_tablon.to_pickle(ruta_df_tablon)